In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch

sns.set_style("whitegrid")
plt.rcParams["font.size"] = 16

In [25]:
pred_mel = torch.rand(1, 4, 8) # (B, C, T)
pred_mel = torch.cat([pred_mel, torch.zeros(1, 4, 8)], dim=2)
pred_mel_reshape = (
    pred_mel.permute(0, 2, 1).reshape(pred_mel.shape[0], -1, 2).permute(0, 2, 1)
)
pred_mel_reshape_2 = pred_mel.reshape(pred_mel.shape[0], 2, -1)
print(pred_mel)
print(pred_mel.permute(0, 2, 1))
print(pred_mel.permute(0, 2, 1).reshape(pred_mel.shape[0], -1, 2))
print(pred_mel_reshape)

tensor([[[0.0084, 0.3036, 0.1252, 0.1386, 0.2088, 0.0429, 0.4710, 0.2439,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.6556, 0.1715, 0.2693, 0.0299, 0.1109, 0.6079, 0.8285, 0.5630,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2591, 0.6451, 0.2668, 0.7009, 0.8179, 0.8899, 0.4067, 0.0550,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4105, 0.1048, 0.7261, 0.1584, 0.2401, 0.5250, 0.3708, 0.1065,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]])
tensor([[[0.0084, 0.6556, 0.2591, 0.4105],
         [0.3036, 0.1715, 0.6451, 0.1048],
         [0.1252, 0.2693, 0.2668, 0.7261],
         [0.1386, 0.0299, 0.7009, 0.1584],
         [0.2088, 0.1109, 0.8179, 0.2401],
         [0.0429, 0.6079, 0.8899, 0.5250],
         [0.4710, 0.8285, 0.4067, 0.3708],
         [0.2439, 0.5630, 0.0550, 0.1065],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.000

In [26]:
print(pred_mel)
print(pred_mel_reshape_2)

tensor([[[0.0084, 0.3036, 0.1252, 0.1386, 0.2088, 0.0429, 0.4710, 0.2439,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.6556, 0.1715, 0.2693, 0.0299, 0.1109, 0.6079, 0.8285, 0.5630,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2591, 0.6451, 0.2668, 0.7009, 0.8179, 0.8899, 0.4067, 0.0550,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4105, 0.1048, 0.7261, 0.1584, 0.2401, 0.5250, 0.3708, 0.1065,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]])
tensor([[[0.0084, 0.3036, 0.1252, 0.1386, 0.2088, 0.0429, 0.4710, 0.2439,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.6556, 0.1715, 0.2693, 0.0299, 0.1109, 0.6079, 0.8285, 0.5630,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2591, 0.6451, 0.2668, 0.7009, 0.8179, 0.8899, 0.4067, 0.0550,
          0.0000, 0.0000, 0.000